In [ ]:
"""
This file is for evaluation experiment  
"""

import numpy as np 
from statsmodels import api as sm
from scipy.fft import fft, ifft, fftfreq

In [ ]:
# Generate samples 
periodic_seed = []
for i in range(99, 10000, 100):
    seq_seed = [1] + [0] * i
    periodic_seed.append(seq_seed)


length = 86400 * 5
periodic_sequences = []
for seq_seed in periodic_seed:
    tmp_seq = [seq_seed[i % len(seq_seed)] for i in range(length)]
    periodic_sequences.append(tmp_seq)
print(len(periodic_sequences))
shuffle_sequences = []
# for i in periodic_sequences:
#     shuffle = np.random.permutation(i).tolist()
#     shuffle_sequences.append(shuffle)
for i in periodic_sequences:
    shuffle = np.random.permutation(periodic_sequences[-1]).tolist()
    shuffle_sequences.append([i[j]|shuffle[j] for j in range(len(i)) ])
print(len(shuffle_sequences))
test_sequences = periodic_sequences + shuffle_sequences
test_sequences = shuffle_sequences
print(len(test_sequences))


In [ ]:
sampling_rate = 1
TP = 0
FN = 0
FP = 0
TN = 0

for seq_seed_index in range(len(test_sequences)):
    print('------')

    cur_sequence = test_sequences[seq_seed_index]
    x = cur_sequence
    y = cur_sequence
    N = len(x) #  total number of discrete data points taken.
    T = N / N
    # sampling frequency 
    f_s = 1/T

    yf = fft(y)
    xf = fftfreq(N, T)[:N//2]
    # yf[0] is dc
    tmp_max = np.max(np.abs(yf[0:N//2]))
    tmp_mean = np.mean(np.abs(yf[0:N//2]))
    tmp_std = np.std(np.abs(yf[0:N//2]))

    p_max_list = []
    for i in range(100):
        y_shuffle = np.random.permutation(y).tolist()
        p_max_list.append(np.max(np.abs(fft(y_shuffle)[1:N//2]).tolist()))
    
    if sampling_rate >= 600:
        threshold_99 = sorted(p_max_list)[-11]
        # print('Threshold 90 percentile: ', threshold_99)
    else:
        threshold_99 = sorted(p_max_list)[-6]
        # print('Threshold 95 percentile: ', threshold_99

    tmp_list = []
    tmp_list_yf = []

    for i in range(len(yf[0:N//2])):
        if i == 0 or i == 1  or i ==len(yf)-1:
            continue
        if np.abs(yf[i]) > threshold_99:
            tmp_list.append(i)
            tmp_list_yf.append(np.abs(yf[i]))


    period = []
    period_tmp_list = []
    if len(tmp_list) >0:
        for i in range(len(tmp_list)):
            if sampling_rate >600 or round(N/tmp_list[i]) >= 10:
                if len(period) == 0:
                    period.append(round(N/tmp_list[i]))
                    period_tmp_list.append(tmp_list[i])
                else:
                    if round(N/tmp_list[i]) != period[-1]:
                        period.append(round(N/tmp_list[i]))
                        period_tmp_list.append(tmp_list[i])


    if seq_seed_index < len(test_sequences)/2:
        period_ground_truth = (seq_seed_index+1)*100
    else:
        period_ground_truth = 0
    print('Sequences %d: period %d ' % (seq_seed_index+1, period_ground_truth))
    print('period:',period[:])
    period_result = []
    if len(period) > 0:
        period_result.append(period[0])

    acf = sm.tsa.acf(y, nlags=len(y),fft=True)

    autocorrelation = []
    if len(period) == 0:
        pass
    else:
        for i in range(len(period)):
            tmp_range = [max(round(N/(period_tmp_list[i]-1)),period[i]+1), min(round(N/(period_tmp_list[i]+1)),period[i]-1)]
            j = tmp_range[0]
            while (j >= tmp_range[1]):
                if j >= len(acf):
                    break
                auto_tmp = acf[j]

                if auto_tmp >= 3.315/np.sqrt(N):
                    autocorrelation.append(((j,auto_tmp))) # '%d:%d ' % 
                j-=1

        autocorrelation = set(autocorrelation)
        autocorrelation = sorted(autocorrelation,key=lambda x:x[1], reverse = True)
        if len(autocorrelation) > 20:
            print('## Autocorrelation ',autocorrelation[:20])
        else:
            print('## Autocorrelation ',autocorrelation)
        if len(autocorrelation) > 0:
            period_result.append(autocorrelation[0][0])

        # print(time_diff)
    if period_ground_truth != 0:
        if period_ground_truth in period_result:
            TP += 1
        else:
            FN += 1
    else:
        if period_ground_truth in period_result:
            FP += 1
        else:
            TN += 1
print('TP:', TP)
print('FN:', FN)
print('FP:', FP)
print('TN:', TN)